In [31]:
#Написать приложение, которое собирает основные новости с сайтов mail.ru, lenta.ru, yandex-новости. 
#Для парсинга использовать XPath. Структура данных должна содержать:
#название источника;
#наименование новости;
#ссылку на новость;
#дата публикации.

import requests
from lxml import html
import re
import datetime
from pymongo import MongoClient
import pymongo.errors
import pandas as pd

header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
        AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 \
        Safari/537.36'}

In [32]:
def mail_news():
    main_link = 'https://news.mail.ru/'

    response = requests.get(main_link, headers=header)
    if response.ok:
        item = html.fromstring(response.text)
        news_links = set(item.xpath(
            '//div[@class="js-module" and @data-module="TrackBlocks"]//a/@href'))
        mail_news = []

        for link in news_links:
            inner_response = requests.get(link, headers=header)
            if inner_response.ok:
                new_item = html.fromstring(inner_response.text)
                news = new_item.xpath(
                    '//div[contains(@class, "js-article")]')
                for n in news:
                    news_data = {}
                    news_data['link'] = link
                    news_data['title'] = n.xpath('.//h1/text()')[0].replace(
                        u'\xa0', ' ')
                    news_data['source'] = n.xpath('.//a//text()')[0]
                    news_data['date'] = \
                        n.xpath('.//*/@datetime')[0].split('T')[0]
                    mail_news.append(news_data)
        return mail_news

In [33]:
def lenta_news():

    main_link = 'https://lenta.ru'
    
    response = requests.get(main_link, headers=header)
    if response.ok:
        item = html.fromstring(response.text)
        news = item.xpath('.//div[@class="first-item" or @class="item"]')
        lenta_news = []
        for n in news:
            news_data = {}
            news_data['title'] = n.xpath("./descendant::a/text()")[0].replace(
                u'\xa0', ' ')
            link_text = n.xpath("./a/@href")[0]
            if 'https' not in link_text:
                news_data['source'] = 'Lenta.ru'
                news_data['link'] = main_link + link_text
                date_list = link_text.strip('/').split('/')[1:4]
                news_data['date'] = '-'.join(date_list[i] for i in range(3))
            else:
                news_data['source'] = link_text.split('/')[2]
                date_list = re.findall(r'\d{2}-\d{2}-\d{4}', link_text)[
                    0].split('-')
                news_data['link'] = link_text
                news_data['date'] = '-'.join(
                    date_list[::-1][i] for i in range(3))
            lenta_news.append(news_data)
        return lenta_news

In [34]:
def yandex_news():

    main_link = 'https://yandex.ru/news'

    response = requests.get(main_link, headers=header)
    if response.ok:
        item = html.fromstring(response.text)
        news = item.xpath(
            '//div[contains(@class, "news-top-flexible-stories" )]/div')
        yandex_news = []
        for n in news:
            news_data = {}
            news_data['title'] = n.xpath(".//h2/text()")[0].replace(u'\xa0',
                                                                    ' ')
            news_data['link'] = \
                n.xpath('.//*[contains(@class, "mg-card__")]/a/@href')[0]
            source_time = n.xpath(
                './/div[@class="mg-card-footer__left"]//text()')
            news_data['source'] = source_time[0]
            today = datetime.date.today()
            news_data['date'] = today.strftime('%Y-%m-%d') if \
                'Вчера' not in source_time[1] else today.replace(
                day=today.day - 1).strftime('%Y-%m-%d')
            yandex_news.append(news_data)
        return yandex_news

In [35]:
def parser_news():

    all_news = []

    all_news.extend(lenta_news())
    all_news.extend(yandex_news())
    all_news.extend(mail_news())
    return all_news
df = pd.DataFrame(news_scrap())
df

,title,source,link,date
0,Россияне раскрыли отношение к половому воспита...,Lenta.ru,https://lenta.ru/news/2021/09/13/materinstvo/,2021-09-13
1,Российский полицейский погиб при попытке остан...,Lenta.ru,https://lenta.ru/news/2021/09/13/amur/,2021-09-13
2,Рухнувший в Иркутской области L-410 нашли с по...,Lenta.ru,https://lenta.ru/news/2021/09/13/kolonka_spasa...,2021-09-13
3,В Подмосковье объявили о начале отопительного ...,moslenta.ru,https://moslenta.ru/news/v-podmoskove-obyavili...,2021-09-13
4,Политолог назвал главное условие встречи Зелен...,Lenta.ru,https://lenta.ru/news/2021/09/13/nuance/,2021-09-13
5,Следы вещества для уничтожения насекомых нашли...,Lenta.ru,https://lenta.ru/news/2021/09/13/rospotreb/,2021-09-13
6,Судан опроверг сообщения о просьбе экономическ...,Lenta.ru,https://lenta.ru/news/2021/09/13/no_help_for_s...,2021-09-13
7,Кремль высказался о ситуации со скифским золотом,Lenta.ru,https://lenta.ru/news/2021/09/13/skolotoi/,2021-09-13
8,Российский боксер-чемпион раскрыл размер своей...,Lenta.ru,https://lenta.ru/news/2021/09/13/provodnikov/,2021-09-13
9,Оценена связь между арбузом и обнаруженным эти...,Lenta.ru,https://lenta.ru/news/2021/09/13/chimik_etilen...,2021-09-13


In [36]:
def add_news():

    client = MongoClient('localhost', 27017)
    base_name = 'top_news'
    db = client[base_name]
    news = db.news
    news.create_index('title', unique=True)
    print(f'База {base_name} обновлена!')
    count = 0  # счетчик уникальных новостей
    total_count = 0  # счетчик скачаных новостей
    for _ in news_scrap():
        try:
            news.insert_one(_)
            count += 1
            total_count += 1
        except pymongo.errors.DuplicateKeyError:
            total_count += 1
            continue
    print(f'Новостей найдено: {total_count}, новостей добавлено: {count}.')


add_news()

База top_news обновлена!
Новостей найдено: 32, новостей добавлено: 3.
